Importando as bibliotecas

In [1]:
import numpy as np
import pandas as pd

Carregando base de dados

In [2]:
# Treinamento com a variável alvo
transact_train_database = pd.read_csv('data/transact_train.txt', sep='|')

# Teste sem a variável alvo
transact_test_database = pd.read_csv('data/transact_class.txt', sep = '|')

# Variável alvo do teste
transact_original_database = pd.read_csv('data/realclass_t1.txt', sep = '|')

Alterando a granularidade

In [3]:
transact_train_database = transact_train_database.drop_duplicates(subset=['sessionNo'], keep='last')
transact_train_database_y = transact_train_database['order']
transact_train_database_y = transact_train_database_y.replace(to_replace=['y', 'n'], value=[1, 0])
transact_train_database_x = transact_train_database.drop(['order'], axis=1)

transact_test_database_x = transact_test_database.drop_duplicates(subset=['sessionNo'], keep='last')

In [4]:
transact_train_database_x.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
2,1,6,5,39.887,1,59.99,59.99,59.99,1,59.99,...,y,completely orderable,1,600,70,21,1,43,1,49
7,2,6,5,94.469,0,?,?,?,0,?,...,y,completely orderable,?,?,?,?,?,?,?,?
12,3,6,5,341.613,11,9.99,29.99,109.95,2,9.99,...,y,completely orderable,3,1800,475,302,12,45,1,11
14,4,6,5,42.812,4,4.99,4.99,19.96,1,4.99,...,y,completely not orderable,?,?,?,?,?,?,?,?
22,5,6,5,2816.046,45,12.99,179.95,1093.72,4,19.99,...,y,completely orderable,4,800,503,18,1,46,1,40


In [5]:
transact_test_database_x.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
8,1,18,7,624.606,11,16.99,39.99,207.91,5,16.99,...,y,completely orderable,25039,1300,489,188,5,49,1,65
19,2,18,7,2804.705,16,34.99,34.99,174.95,2,34.99,...,y,completely orderable,25040,1200,543,43,5,29,2,184
42,3,18,7,7401.384,119,7.99,59.95,3263.57,12,12.49,...,y,completely orderable,25041,600,552,17,4,37,2,107
48,4,18,7,2853.550,152,3.99,239.99,5642.5,4,9.99,...,?,?,25042,8500,535,226,19,49,2,17
50,5,18,7,48.145,2,29.99,29.99,59.98,1,29.99,...,y,completely orderable,25043,600,543,39,2,53,2,234


In [6]:
transact_train_database_y.head()

2     1
7     1
12    1
14    0
22    1
Name: order, dtype: int64

Tratando os valores ausentes

Lista de variáveis numéricas

In [7]:
list_numeric_variables = [
  'cMinPrice',
  'cMaxPrice',
  'cSumPrice',
  'bMinPrice',
  'bMaxPrice',
  'bSumPrice',
  'bStep',
  'maxVal',
  'customerScore',
  'accountLifetime',
  'payments',
  'age',
  'address',
  'lastOrder',
]

Lista de variáveis string

In [8]:
list_string_variables = [
  'availability',
  'onlineStatus',
]

In [9]:
def replaceValueMissing(df, columns, value):
  removed = df[columns].replace('?', value)
  for column in columns:
    df[column] = removed[column]

  return df

In [10]:
def convertFloat(df, columns):
  for column in columns:
    df[column] = df[column].astype(float)

  return df

In [11]:
def replaceMissingByMean(df, columns):
  for column in columns:
    df[column] = df[column].replace(to_replace='?', value=df[column].mean())

  return df

In [12]:
def replaceMissingByMode(df, columns):
  for column in columns:
    df[column] = df[column].replace(to_replace='?', value=df[column].mode()[0])

  return df

In [13]:
def replaceMissingByFrequency(df, columns):
  for column in columns:
    df[column] = df[column].replace(to_replace='?', value=df[column].value_counts().index[0])

  return df

In [14]:
def replaceMissingByMedian(df, columns):
  for column in columns:
    df[column] = df[column].replace(to_replace='?', value=df[column].median())

  return df

In [15]:
def replaceMissingByFixedValue(df, columns, value):
  for column in columns:
    df[column] = df[column].replace(to_replace='?', value=value)

  return df

In [16]:
transact_train_database_x.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
2,1,6,5,39.887,1,59.99,59.99,59.99,1,59.99,...,y,completely orderable,1,600,70,21,1,43,1,49
7,2,6,5,94.469,0,?,?,?,0,?,...,y,completely orderable,?,?,?,?,?,?,?,?
12,3,6,5,341.613,11,9.99,29.99,109.95,2,9.99,...,y,completely orderable,3,1800,475,302,12,45,1,11
14,4,6,5,42.812,4,4.99,4.99,19.96,1,4.99,...,y,completely not orderable,?,?,?,?,?,?,?,?
22,5,6,5,2816.046,45,12.99,179.95,1093.72,4,19.99,...,y,completely orderable,4,800,503,18,1,46,1,40


In [17]:
transact_test_database_x.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
8,1,18,7,624.606,11,16.99,39.99,207.91,5,16.99,...,y,completely orderable,25039,1300,489,188,5,49,1,65
19,2,18,7,2804.705,16,34.99,34.99,174.95,2,34.99,...,y,completely orderable,25040,1200,543,43,5,29,2,184
42,3,18,7,7401.384,119,7.99,59.95,3263.57,12,12.49,...,y,completely orderable,25041,600,552,17,4,37,2,107
48,4,18,7,2853.550,152,3.99,239.99,5642.5,4,9.99,...,?,?,25042,8500,535,226,19,49,2,17
50,5,18,7,48.145,2,29.99,29.99,59.98,1,29.99,...,y,completely orderable,25043,600,543,39,2,53,2,234


Base na granularidade do projeto

In [18]:
# Substituindo a string ? por np.nan em todas p/ todas as variáveis da base na granularidade do projeto

transact_train_database_x = replaceValueMissing(transact_train_database_x, list_numeric_variables, np.nan)
transact_train_database_x = replaceValueMissing(transact_train_database_x, list_string_variables, np.nan)

transact_test_database_x = replaceValueMissing(transact_test_database_x, list_numeric_variables, np.nan)
transact_test_database_x = replaceValueMissing(transact_test_database_x, list_string_variables, np.nan)

transact_train_database_x = convertFloat(transact_train_database_x, list_numeric_variables)
transact_test_database_x = convertFloat(transact_test_database_x, list_numeric_variables)

# Substituindo os missing values das variáveis numéricas pela média
transact_train_database_x = replaceMissingByMean(transact_train_database_x, list_numeric_variables)
transact_test_database_x = replaceMissingByMean(transact_test_database_x, list_numeric_variables)

# Substituindo os missing values das variáveis String por um valor fixo
transact_train_database_x = replaceMissingByFixedValue(transact_train_database_x, list_string_variables, 'ausente')
transact_test_database_x = replaceMissingByFixedValue(transact_test_database_x, list_string_variables, 'ausente')

/home/uendel/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [19]:
transact_train_database_x.sample(3)

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
19789,2408,9,5,1977.972,24,11.99,34.99,263.89,3,11.99,...,y,completely orderable,1210,4000.0,543.0,190.0,16.0,36.0,2.0,18.0
126517,14975,20,5,2445.054,16,6.99,59.99,331.90,2,9.99,...,y,completely orderable,?,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12851,1648,9,5,8.221,1,6.99,6.99,6.99,1,6.99,...,y,completely orderable,?,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Atribuindo valor zero para as variáveis comportamentais que não possuem valor de customerNo

rows_without_customerNo = transact_train_database_x['customerNo'] == '?'
transact_train_database_x.loc[rows_without_customerNo, 'maxVal'] = 0
transact_train_database_x.loc[rows_without_customerNo, 'customerScore'] = 0
transact_train_database_x.loc[rows_without_customerNo, 'accountLifetime'] = 0
transact_train_database_x.loc[rows_without_customerNo, 'payments'] = 0
transact_train_database_x.loc[rows_without_customerNo, 'age'] = 0
transact_train_database_x.loc[rows_without_customerNo, 'address'] = 0
transact_train_database_x.loc[rows_without_customerNo, 'lastOrder'] = 0

# Teste
rows_test_without_customerNo = transact_test_database_x['customerNo'] == '?'
transact_test_database_x.loc[rows_test_without_customerNo, 'maxVal'] = 0
transact_test_database_x.loc[rows_test_without_customerNo, 'customerScore'] = 0
transact_test_database_x.loc[rows_test_without_customerNo, 'accountLifetime'] = 0
transact_test_database_x.loc[rows_test_without_customerNo, 'payments'] = 0
transact_test_database_x.loc[rows_test_without_customerNo, 'age'] = 0
transact_test_database_x.loc[rows_test_without_customerNo, 'address'] = 0
transact_test_database_x.loc[rows_test_without_customerNo, 'lastOrder'] = 0

/home/uendel/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [21]:
# Substituindo a string ? por np.nan em todas p/ todas as variáveis da base a base original
transact_test_database = replaceValueMissing(transact_test_database.copy(), list_numeric_variables, np.nan)
transact_test_database = replaceValueMissing(transact_test_database.copy(), list_string_variables, np.nan)
transact_train_database = replaceValueMissing(transact_train_database.copy(), list_numeric_variables, np.nan)
transact_train_database = replaceValueMissing(transact_train_database.copy(), list_string_variables, np.nan)

transact_train_database = convertFloat(transact_train_database, list_numeric_variables)
transact_test_database = convertFloat(transact_test_database, list_numeric_variables)

In [22]:
transact_train_database_x.describe(include='all')

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,49682.000000,49682.000000,49682.000000,50000.000000,49325.000000,...,39027,37219,50000,49685.000000,49685.000000,49685.000000,50000.000000,49972.000000,50000.000000,50000.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,7,25038,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,y,completely orderable,?,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38684,36277,22639,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,25000.500000,14.367060,5.923140,1465.728502,20.233700,72.118160,157.646205,1096.642791,3.207580,86.444819,...,NaN,NaN,NaN,1233.813022,263.318990,69.768884,7.055600,24.570299,0.934060,43.234580
std,14433.901067,4.638547,0.795063,2314.980566,28.430273,176.181174,295.331281,3083.223860,3.549983,203.739233,...,NaN,NaN,NaN,2325.130289,259.689366,102.483938,20.798151,24.080901,0.914576,92.009748
min,1.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12500.750000,11.000000,5.000000,217.926750,4.000000,7.000000,24.990000,119.960000,1.000000,9.990000,...,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,25000.500000,15.000000,6.000000,686.784500,10.000000,14.990000,49.990000,358.935000,2.000000,19.990000,...,NaN,NaN,NaN,300.000000,80.000000,6.000000,0.000000,28.000000,1.000000,7.000000
75%,37500.250000,18.000000,7.000000,1726.262750,24.000000,39.990000,129.990000,938.637500,4.000000,49.990000,...,NaN,NaN,NaN,1500.000000,525.000000,114.000000,9.000000,46.000000,2.000000,38.000000


In [23]:
transact_test_database_x.describe(include='all')

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
count,5111.000000,5111.000000,5111.00000,5111.000000,5111.000000,5079.000000,5079.000000,5079.000000,5111.000000,5042.000000,...,3917,3771,5111,5079.000000,5079.000000,5079.000000,5111.000000,5107.000000,5111.000000,5111.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,6,2441,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,y,completely orderable,?,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3893,3694,2538,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2556.000000,17.996087,6.23107,1527.974712,23.304637,72.880350,171.776058,1267.255050,3.395813,89.261148,...,NaN,NaN,NaN,965.071865,238.451467,60.392794,4.780669,21.894654,0.862062,44.962434
std,1475.562943,6.365912,2.00577,2184.293690,31.657279,167.712913,304.336070,3753.375477,3.641097,195.863501,...,NaN,NaN,NaN,1758.659650,258.839608,95.620631,10.691686,23.337228,0.915378,97.782484
min,1.000000,0.000000,1.00000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000,0.000000,...,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1278.500000,19.000000,7.00000,206.114500,4.000000,6.990000,29.990000,134.905000,1.000000,9.225000,...,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2556.000000,20.000000,7.00000,793.389000,11.000000,12.990000,49.990000,399.970000,2.000000,17.990000,...,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,20.000000,1.000000,4.000000
75%,3833.500000,21.000000,7.00000,1966.657000,30.000000,44.990000,159.990000,1089.295000,4.000000,49.990000,...,NaN,NaN,NaN,1000.000000,520.000000,87.500000,7.000000,44.000000,2.000000,37.500000


In [24]:
transact_train_database.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order
2,1,6,5,39.887,1,59.99,59.99,59.99,1,59.99,...,completely orderable,1,600.0,70.0,21.0,1.0,43.0,1.0,49.0,y
7,2,6,5,94.469,0,NaN,NaN,NaN,0,NaN,...,completely orderable,?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,y
12,3,6,5,341.613,11,9.99,29.99,109.95,2,9.99,...,completely orderable,3,1800.0,475.0,302.0,12.0,45.0,1.0,11.0,y
14,4,6,5,42.812,4,4.99,4.99,19.96,1,4.99,...,completely not orderable,?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,n
22,5,6,5,2816.046,45,12.99,179.95,1093.72,4,19.99,...,completely orderable,4,800.0,503.0,18.0,1.0,46.0,1.0,40.0,y


In [25]:
transact_test_database.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
0,1,18,7,136.833,3,39.99,39.99,79.98,1,39.99,...,y,completely orderable,25039,1300.0,489.0,188.0,5.0,49.0,1.0,65.0
1,1,18,7,189.984,3,39.99,39.99,79.98,1,39.99,...,y,completely orderable,25039,1300.0,489.0,188.0,5.0,49.0,1.0,65.0
2,1,18,7,342.894,6,16.99,39.99,113.96,2,16.99,...,NaN,NaN,25039,1300.0,489.0,188.0,5.0,49.0,1.0,65.0
3,1,18,7,411.051,8,16.99,39.99,149.94,3,16.99,...,NaN,NaN,25039,1300.0,489.0,188.0,5.0,49.0,1.0,65.0
4,1,18,7,460.049,10,16.99,39.99,189.92,4,16.99,...,NaN,NaN,25039,1300.0,489.0,188.0,5.0,49.0,1.0,65.0


Criando novas variáveis

In [26]:
transact_train_database_x = transact_train_database_x.set_index('sessionNo')
transact_test_database_x = transact_test_database_x.set_index('sessionNo')

In [27]:
# Criando variável bSumPrice_cSumPrice
transact_test_database_x['bSumPrice_cSumPrice'] = transact_test_database_x['bSumPrice'] / transact_test_database_x['cSumPrice']
transact_train_database_x['bSumPrice_cSumPrice'] = transact_train_database_x['bSumPrice'] / transact_train_database_x['cSumPrice']

# --------- Definindo lastOrder_accountLifetime ---------


In [28]:
# Criando variável lastOrder_accountLifetime
transact_test_database_x['lastOrder_accountLifetime'] = transact_test_database_x['lastOrder'] / transact_test_database_x['accountLifetime']
transact_train_database_x['lastOrder_accountLifetime'] = transact_train_database_x['lastOrder'] / transact_train_database_x['accountLifetime']


In [29]:
# Criando variável bCount_cCount
transact_test_database_x['bCount_cCount'] = transact_test_database_x['bCount'] / transact_test_database_x['cCount']
transact_train_database_x['bCount_cCount'] = transact_train_database_x['bCount'] / transact_train_database_x['cCount']


In [44]:
transact_test_database_x.head()

,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,bMaxPrice,...,lastOrder_accountLifetime,bCount_cCount,__n,__y,__completely not determinable,__completely not orderable,__completely orderable,__mainly not orderable,__mainly orderable,__mixed
sessionNo,,,,,,,,,,,,,,,,,,,,,
1,18,7,624.606,11,16.99,39.99,207.91,5,16.99,39.99,...,0.345745,0.454545,0,1,0,0,1,0,0,0
2,18,7,2804.705,16,34.99,34.99,174.95,2,34.99,34.99,...,4.279070,0.125000,0,1,0,0,1,0,0,0
3,18,7,7401.384,119,7.99,59.95,3263.57,12,12.49,39.95,...,6.294118,0.100840,0,1,0,0,1,0,0,0
4,18,7,2853.550,152,3.99,239.99,5642.50,4,9.99,14.99,...,0.075221,0.026316,0,0,0,0,0,0,0,0
5,18,7,48.145,2,29.99,29.99,59.98,1,29.99,29.99,...,6.000000,0.500000,0,1,0,0,1,0,0,0


In [45]:
transact_train_database_x.head()

,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,bMaxPrice,...,bCount_cCount,__n,__y,__completely not determinable,__completely not orderable,__completely orderable,__mainly not determinable,__mainly not orderable,__mainly orderable,__mixed
sessionNo,,,,,,,,,,,,,,,,,,,,,
1,6,5,39.887,1,59.99,59.99,59.99,1,59.99,59.99,...,1.000000,0,1,0,0,1,0,0,0,0
2,6,5,94.469,0,NaN,NaN,NaN,0,NaN,NaN,...,NaN,0,1,0,0,1,0,0,0,0
3,6,5,341.613,11,9.99,29.99,109.95,2,9.99,29.99,...,0.181818,0,1,0,0,1,0,0,0,0
4,6,5,42.812,4,4.99,4.99,19.96,1,4.99,4.99,...,0.250000,0,1,0,1,0,0,0,0,0
5,6,5,2816.046,45,12.99,179.95,1093.72,4,19.99,27.85,...,0.088889,0,1,0,0,1,0,0,0,0


# Transformação dos Dados

In [32]:
#  Remove variáveis contendo ids
transact_train_database_x = transact_train_database_x.drop('customerNo', axis=1)
transact_test_database_x = transact_test_database_x.drop('customerNo', axis=1)

In [33]:
# Criação de variáveis dummies
transact_train_database_x = pd.get_dummies(transact_train_database_x, prefix='_')
transact_test_database_x = pd.get_dummies(transact_test_database_x, prefix='_')

In [58]:
assert(transact_train_database_x.shape[1] == transact_test_database_x.shape[1]), 'Número de variáveis diferente'

AssertionError: Número de variáveis diferente

In [63]:
variables_transact_train_database_x = list(transact_train_database_x.columns)
variables_transact_test_database_x = list(transact_test_database_x.columns)

in_train_not_test = [i for i in variables_transact_train_database_x if i not in variables_transact_test_database_x]
in_test_not_train = [i for i in variables_transact_test_database_x if i not in variables_transact_train_database_x]

if len(in_train_not_test) > 0:
    print('Variáveis que tão no treinamneto e não no teste')
    print(in_train_not_test)
else:
    print('Todas as variáveis do treinamento estão no teste')

if len(in_test_not_train) > 0:
    print('Variáveis que tão no teste e não no treinamento')
    print(in_test_not_train)
else:
    print('Todas as variáveis do teste estão no treinamento')

Todas as variáveis do treinamento estão no teste
Todas as variáveis do teste estão no treinamento


In [61]:
transact_train_database_x['__mainly not determinable'] = transact_train_database_x['__mainly not determinable'] + transact_train_database_x['__mainly not determinable']
transact_train_database_x = transact_train_database_x.drop('__mainly not determinable', axis=1)

In [64]:
assert(transact_train_database_x.shape[1] == transact_test_database_x.shape[1]), 'Número de variáveis diferente'

In [66]:
# Checando se todas as colunas estão como numericas, ou seja, não existem variáveis String (type object)
columns_obj = transact_train_database_x.dtypes[transact_train_database_x.dtypes == np.object]

assert(len(columns_obj) == 0), 'Existem variáveis do tipo object'

columns_obj_test = transact_test_database_x.dtypes[transact_test_database_x.dtypes == np.object]

assert(len(columns_obj_test) == 0), 'Existem variáveis do tipo object'

print('Passou no teste: as bases não possuem variáveis String!')

Passou no teste: as bases não possuem variáveis String!


# Salvando as bases de dados

In [ ]:
transact_train_database_x.to_csv('transact_train_database_x.csv', index=False)
transact_test_database_x.to_csv('transact_test_database_x.csv', index=False)